In [1]:
!git clone https://github.com/sameerkhan7/Robot_Tool_Segmentation.git
%cd Robot_Tool_Segmentation


fatal: destination path 'Robot_Tool_Segmentation' already exists and is not an empty directory.
/content/Robot_Tool_Segmentation


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
!pwd

/content/Robot_Tool_Segmentation


# Downloading the data ONCE

In [4]:
# # Downloading
# !cp /content/drive/MyDrive/data.zip /content/Robot_Tool_Segmentation/


In [5]:
%cd /content/Robot_Tool_Segmentation/
# !unzip data.zip -d data

/content/Robot_Tool_Segmentation


In [6]:
#%cd /content/Robot_Tool_Segmentation
# Remove the 'data' folder and its contents
# !rm -rf data


In [7]:
# Libraries
!ls
!pwd

BAAnet.ipynb  data		EncoderModule.py      model.pth    TrainingUtils.py
BAAnet.py     data.zip		EndovisDataloader.py  __pycache__
BBAModule.py  DecoderModule.py	LICENSE		      README.md
/content/Robot_Tool_Segmentation


In [8]:
import torch
import random
import matplotlib.pyplot as plt
import numpy as np
from tqdm.notebook import tqdm
import EndovisDataloader
import BBAModule
import EncoderModule
import TrainingUtils
import BAAnet
import DecoderModule


In [9]:
# set seed for reprodicibility
def set_seed(seed):
    random.seed(seed)  # Python random seed
    np.random.seed(seed)  # NumPy random seed
    torch.manual_seed(seed)  # PyTorch random seed for CPU
    torch.cuda.manual_seed(seed)  # PyTorch random seed for GPU
    torch.cuda.manual_seed_all(seed)  # PyTorch random seed for all GPUs
    # Ensure deterministic behavior in CUDA
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

In [10]:
%cd /content/Robot_Tool_Segmentation/data/data
!pwd


/content/Robot_Tool_Segmentation/data/data
/content/Robot_Tool_Segmentation/data/data


In [11]:
import os

In [12]:
os.getcwd()

'/content/Robot_Tool_Segmentation/data/data'

In [13]:
%cd ../..

/content/Robot_Tool_Segmentation


In [18]:
# get dataloaders and display test images
seed = 42
batch_size = 32
reduce_factor = 4 # how much to reduce image size by before training
num_workers = 0 # 0 for notebooks
train_dataloader, val_dataloader, test_dataloader = EndovisDataloader.getDataloaders(batch_size, reduce_factor, num_workers,seed)


['/content/Robot_Tool_Segmentation/data/data/seq_1/frames', '/content/Robot_Tool_Segmentation/data/data/seq_10/frames', '/content/Robot_Tool_Segmentation/data/data/seq_11/frames', '/content/Robot_Tool_Segmentation/data/data/seq_12/frames', '/content/Robot_Tool_Segmentation/data/data/seq_13/frames', '/content/Robot_Tool_Segmentation/data/data/seq_14/frames', '/content/Robot_Tool_Segmentation/data/data/seq_15/frames', '/content/Robot_Tool_Segmentation/data/data/seq_16/frames', '/content/Robot_Tool_Segmentation/data/data/seq_2/frames', '/content/Robot_Tool_Segmentation/data/data/seq_3/frames', '/content/Robot_Tool_Segmentation/data/data/seq_4/frames', '/content/Robot_Tool_Segmentation/data/data/seq_5/frames', '/content/Robot_Tool_Segmentation/data/data/seq_6/frames', '/content/Robot_Tool_Segmentation/data/data/seq_7/frames', '/content/Robot_Tool_Segmentation/data/data/seq_9/frames', '/content/Robot_Tool_Segmentation/data/data/test_seq_1/frames', '/content/Robot_Tool_Segmentation/data/data

## Setting Hyperparameters

In [20]:
# model and hyperparameters
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
epochs = 50
lr = 1e-4
set_seed(seed) # set random seed
#model = BAAnet.BAANet_BBA_Only().to(device)
model = BAAnet.BAANet().to(device)
#model = BAAnet.BAANet_BAF_Only().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=lr) # Adam optimizer
criterion = TrainingUtils.DICELoss # loss function
#criterion = torch.nn.CrossEntropyLoss()
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=50, gamma=0.5) # learning rate scheduler

cuda:0


## Training

In [21]:
# train model. Output is [epoch, training_loss, mdice, miou, mdice_robot, miou_robot]
training_outputs = []

for i in range(epochs):
    training_output = TrainingUtils.train(train_dataloader, val_dataloader, model, criterion, optimizer, scheduler, i, device)
    training_outputs.append(training_output)

52it [02:42,  3.12s/it]
5it [03:26, 41.26s/it]


KeyboardInterrupt: 

In [ ]:
# save trained model
torch.save(model.state_dict(), "model1.pth")

## Results and Plots

In [ ]:
# process outputs 
training_outputs_cpu = []
for i in training_outputs:
    temp = []
    for j in i:
        if torch.is_tensor(j):
            temp.append(j.cpu())
        else:
            temp.append(j)
    training_outputs_cpu.append(temp)
    
training_outputs_cpu = np.array(training_outputs_cpu)

#print training loss
plt.figure(figsize=(5, 5))
plt.plot(training_outputs_cpu[:,0], training_outputs_cpu[:,1], label='Training Loss')
plt.title('Training Loss over Epochs')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid()
plt.show()

#print validation scores
training_outputs_cpu_filtered = [value for index, value in enumerate(training_outputs_cpu) if index % 5 == 0]
training_outputs_cpu_filtered = np.array(training_outputs_cpu_filtered)
print(training_outputs_cpu_filtered)
plt.figure(figsize=(5, 5))
plt.plot(training_outputs_cpu_filtered[:,0], training_outputs_cpu_filtered[:,2], label='mDICE')
plt.plot(training_outputs_cpu_filtered[:,0], training_outputs_cpu_filtered[:,3], label='mIoU')
plt.plot(training_outputs_cpu_filtered[:,0], training_outputs_cpu_filtered[:,4], label='mDICE Robot')
plt.plot(training_outputs_cpu_filtered[:,0], training_outputs_cpu_filtered[:,5], label='mIoU Robot')
plt.title('Validation Accuracy over Epochs')
plt.xlabel('Epoch')
plt.ylabel('Validation Accuracy')
plt.legend()
plt.grid()
plt.show()

In [ ]:
# Test Model on Test Data
# test model
m_dice, m_iou, mdice_robot, miou_robot = TrainingUtils.test(model, test_dataloader, device)
print(f"Mean DICE Score: {m_dice}")
print(f"Mean IoU Score: {m_iou}")
print(f"Mean DICE Robot Score: {mdice_robot}")
print(f"Mean IoU Robot Score: {miou_robot}")

In [ ]:
def get_ith_sample(dataloader, i):
    
    batch_size = dataloader.batch_size  
    batch_idx = i // batch_size         
    sample_idx = i % batch_size         

    for current_batch_idx, (images, labels) in enumerate(dataloader):
        if current_batch_idx == batch_idx:
            # Return the i-th sample from the batch
            return images[sample_idx], labels[sample_idx]

# display model outputs
model.cpu()
#i = 1 # 0 to batch_size
# pick 3 "random" images to compare models on
imgs, labels = [], []
img1, lab1 = get_ith_sample(val_dataloader, 198)
img2, lab2 = get_ith_sample(val_dataloader, 271)
img3, lab3 = get_ith_sample(test_dataloader, 159)
imgs = [img1, img2, img3]
labels = [lab1, lab2, lab3]
results = []

for i in range(3):
    input, label = imgs[i].unsqueeze(0), labels[i].unsqueeze(0)

    predict = model(input) # predict image
    predict = torch.argmax(predict, dim=1) # get grayscale prediction
    predict = torch.eye(12, device=predict.device)[predict].permute(0,3,1,2)
    predict = torch.nn.functional.interpolate(predict.float(), size=label.shape[2:], mode="nearest-exact")  # match label size
    prediction = EndovisDataloader.convert_masks_to_gray(predict.squeeze(0))

    label = EndovisDataloader.convert_masks_to_gray(label.squeeze(0))

    cmap = plt.get_cmap('Paired',12)
    cbar_ticks = [0,1,2,3,4,5,6,7,8,9,10,11]
    cbar_labels = ["Background Tissue", "Instrument Shaft", "Instrument Clasper", "Instrument Wrist", "Kidney Parenchyma", "Covered Kidney", 
                "Thread", "Clamps", "Suturing Needle", "Suction Instrument", "Small Intestine", "Ultrasound Probe"]

    # un-normalize input
    mean = torch.tensor([0.464, 0.341, 0.363]).view(-1, 1, 1)
    std = torch.tensor([0.210, 0.194, 0.204]).view(-1, 1, 1)
    input = input.squeeze(0) * std + mean

    results.append((input,label,prediction))


fig, axes = plt.subplots(3, 3, figsize=(10, 10))
for i in range(3):
    axes[i,0].imshow(results[i][0].permute(1, 2, 0))
    axes[i,1].imshow(results[i][1], cmap = cmap, vmin = 0, vmax = 11)
    im = axes[i,2].imshow(results[i][2], cmap = cmap, vmin = 0, vmax = 11)

    cbar = fig.colorbar(im, ax=axes[i,2], fraction=0.046, pad=0.04)
    cbar.set_label("Segmentation Labels", fontsize=10)
    cbar.set_ticks(cbar_ticks)
    cbar.set_ticklabels(cbar_labels)


plt.tight_layout()
plt.show()

# things to record: epochs, batch_size, reduce_factor, training loss curve, training time, validation score curves, test scores, test time, example image (below)